# (PSL) Project 3

Members:
- Amy Hwang (ahwang22)
- Christian Tam (cmtam2)
- Monil Kaneria (kaneria2)

Amy Hwang worked on the

# Section 1

#### Discuss the technical details of the sentiment classification model, including data preprocessing and other key aspects of your model implementation. Your explanation should be detailed enough for your PSL classmates to replicate your results.

We used GridSearchCV for hyperparameter selection. The penalty is elasticnet, and saga is the solver. We used a max iteration count of 2000 to ensure convergence due to the large dataset size.

#### Report the AUC of your predictions on each of the 5 test datasets (refer to the evaluation metric described above), the execution time of your code, and the specifications of the computer system used (e.g., Macbook Pro, 2.53 GHz, 4GB memory, or AWS t2.large) for each of the 5 splits.

Split 1: 
- AUC Score for LogisticRegression: 0.9870942
- Execution time: 30.1842 seconds
  
Split 2:
- AUC Score for LogisticRegression: 0.9867907
- Execution time: 30.1352 seconds
  
Split 3:
- AUC Score for LogisticRegression: 0.9864187
- Execution time: 30.8156 seconds
  
Split 4:
- AUC Score for LogisticRegression: 0.9869783
- Execution time: 31.2468 seconds
  
Split 5:
- AUC Score for LogisticRegression: 0.9862662
- Execution time: 31.1201 seconds

Computer system specs: Macbook Pro, 3.2 Ghz, 32 GB memory

# Section 2

#### Provide a detailed explanation of your interpretability approach.

Obtain BERT embeddings

In [26]:
import pandas as pd
import random
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LinearRegression

In [28]:
test_file_path = f'./F24_Proj3_data/split_1/test.csv'
test_y_file_path = f'./F24_Proj3_data/split_1/test_y.csv'

test = pd.read_csv(test_file_path)
test_y = pd.read_csv(test_y_file_path)

In [29]:
random.seed(42)

Grab 5 random positive reviews and 5 random negative reviews and combine them into one DataFrame.

In [30]:
positive_reviews = test[test_y['sentiment'] == 1].sample(5, random_state=42)
negative_reviews = test[test_y['sentiment'] == 0].sample(5, random_state=42)
selected_reviews = pd.concat([positive_reviews, negative_reviews])

Pull in the BERT embedded data from GitHub.

In [35]:
url = 'https://github.com/hwangsamy1/CS598-PSL/blob/12dec4b55bca444796843097ad0f55df09a74d30/Project3/SBERT_embeddings_random.csv'

In [38]:
pd.read_csv(url, quotechar='"')

ParserError: Error tokenizing data. C error: Expected 1 fields in line 42, saw 22
